In [105]:
import pprint
from h3 import h3
import folium
import pandas as pd

In [106]:
def visualize_hexagons(hexagons, color="red", folium_map=None):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
        m.add_child(my_PolyLine)
    return m

def visualize_points(points,color="blue", folium_map=None):
    if folium_map is None:
        m = folium.Map(location=[points[0][0], points[0][1]], zoom_start=13, tiles='cartodbpositron')
    else:
        m = folium_map
    
    for point in points:
        my_point = folium.Marker([point[0], point[1]], icon=folium.Icon(color=color))
        m.add_child(my_point)

    return m

def visualize_lines(lines,color="black", folium_map=None):
    if folium_map is None:
        m = folium.Map(location=[lines[0][0], lines[0][1]], zoom_start=13, tiles='cartodbpositron')
    else:
        m = folium_map

    opacity = 1.0
    for line in lines:
        my_PolyLine=folium.PolyLine(locations=line,weight=8,color=color, opacity=opacity)
        m.add_child(my_PolyLine)
        opacity -= 0.4
    return m

In [107]:
def clac_euclid_distances(lat,lon,ring_points) -> list:
    euclid_distances = []
    for point in ring_points:
        # ユークリッド距離を調べる
        lat_diff = lat - point[0]
        lon_diff = lon - point[1]
        euclid_distance = (lat_diff**2 + lon_diff**2)**0.5
        euclid_distances.append(euclid_distance)
    return euclid_distances


In [108]:
lat = 35.1705901
lon = 136.8816376

In [109]:
class plot_calc_current_point:
    def __init__(self, mode, lat, lon):
        self.lat = lat
        self.lon = lon
        self.point_colot = "red"
        self.h3_address = h3.geo_to_h3(lat, lon, 9) # lat, lng, hex resolution
        self.current_points = [h3.h3_to_geo(point) for point in h3.k_ring_distances(self.h3_address, 1)[0]]
        if mode == "user":
            self.lat = lat
            self.lon = lon
            self.point_colot = "red"
        else:
            self.lat = self.current_points[0][0]
            self.lon = self.current_points[0][1]
            self.point_colot = "orange"

    def print(self):
        m = folium.Map(location=[self.lat, self.lon], zoom_start=16, tiles='cartodbpositron')
        m = visualize_hexagons(list(h3.k_ring_distances(self.h3_address, 1)[1]), color="green", folium_map=m)
        m = visualize_hexagons(list(h3.k_ring_distances(self.h3_address, 1)[0]), color="red", folium_map=m)
        ring_points = [h3.h3_to_geo(point) for point in h3.k_ring_distances(self.h3_address, 1)[1]]
        m = visualize_points(ring_points, folium_map=m)
        m = visualize_points([[self.lat, self.lon]], color=self.point_colot, folium_map=m)
        euclid_distances = clac_euclid_distances(self.lat, self.lon, ring_points)
        closest_points = [ring_points[euclid_distances.index(dist)] for dist in sorted(euclid_distances)[:2]]
        lines = [[[self.lat, self.lon], closest_point] for closest_point in closest_points]
        m = visualize_lines(lines, folium_map=m)
        display(m)

In [110]:
plot_calc_current_point_user = plot_calc_current_point("user", lat, lon)
plot_calc_current_point_user.print()

In [111]:
plot_calc_current_point_user = plot_calc_current_point("", lat, lon)
plot_calc_current_point_user.print()